# Sequential Chats and Customer Onboarding with AutoGen

This notebook simulates mirrors a real-world customer onboarding pipeline using **AutoGen’s `ConversableAgent` and `initiate_chats`**. Rather than using a single, monolithic conversation, the onboarding process is modularized into sequential, specialized tasks—each handled by a dedicated agent.

## **Human-in-the-loop Interaction**

This workflow also supports **human input mode**, allowing users to type live responses directly into the conversation. The conversation proceeds when the human provides input, or the loop ends if a termination condition (like typing “exit”) is met.

- The **`customer_proxy_agent`** is configured with `human_input_mode="ALWAYS"`, enabling manual input when interacting with other agents.


## Setup

In [5]:
import os

llm_config = {
    "config_list": [{
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"], 
    }]
}

In [ ]:
from autogen import ConversableAgent

## Defining AutoGen Agents

**Each agent is configured with:**
- a unique name
- a system message describing its behavior
- `llm_config` as the language model backend
- `human_input_mode` and optional termination logic using `lambda` conditions
---
and tailored for a specific sub-task:
- **`onboarding_personal_information_agent`**: Gathers the customer's name and location.
- **`onboarding_topic_preference_agent`**: Asks about the customer’s topic preferences.
- **`customer_engagement_agent`**: Provides relevant and engaging information based on prior responses.
- **`customer_proxy_agent`**: Represents the customer interface.

In [6]:
onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [9]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating Tasks

The tasks are structured using a list of dictionaries:

```python
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information into a JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history": True,
    },
    ...
]
```

- Each chat specifies the initiator, recipient, and prompt.
- `summary_method` defines how the result is saved for downstream use.
- `max_turns` limits the number of exchanges.
- `clear_history` controls memory between tasks.

In [10]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process
This triggers the full sequence of tasks. Conversations run in the specified order and summaries from each chat are implicitly passed along. 

In [11]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/python/3.12.1/lib/python3.12/site-packages/autogen/agentchat/chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Mia USA


customer_proxy_agent (to onboarding_personal_information_agent):

Mia USA

--------------------------------------------------------------------------------
onboarding_personal_information_agent (to customer_proxy_agent):

Thank you, Mia from the USA! Your onboarding is complete. If you need any further assistance, feel free to reach out. TERMINATE

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> TERMINATING RUN (7dbd6e13-67ca-489a-ba12-e6e4c8701e5a): Termination message condition on agent 'customer_proxy_agent' met

>>>>>>>> TERMINATING RUN (b574895d-e4c9-44d5-bed8-f1a4ee873026): Maximum turns (2) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
```json
{
  "name": "Mia",
  "location": "USA"
}
```

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  philosophy


customer_proxy_agent (to onboarding_topic_preference_agent):

philosophy

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b430e416-c206-4b8d-b31a-f38b068dcdd5): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
```json
{
  "name": "Mia",
  "location": "USA"
}
```
Mia, from the USA, is interested in reading about philosophy.

--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Hello Mia! Philosophical musings can be quite the mental workout, but I promise this will be a fun little journey. Did you know that ancient Greek philosophers could get pretty quirky? Let me share with you a fun fac

## Chat Summary

In [12]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "name": "Mia",
  "location": "USA"
}
```


Mia, from the USA, is interested in reading about philosophy.


Mia, who resides in the USA, is interested in fun philosophical content, particularly about ancient philosophers like Diogenes of Sinope, known for his eccentric behavior and humorous tales that challenge social values.




## Cost

Token-level and cost breakdown per interaction are provided, which can be useful for tracking API expenses and optimizing prompts.

In [13]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0010250000000000003, 'gpt-4o-2024-08-06': {'cost': 0.0010250000000000003, 'prompt_tokens': 214, 'completion_tokens': 49, 'total_tokens': 263}}, 'usage_excluding_cached_inference': {'total_cost': 0.0010250000000000003, 'gpt-4o-2024-08-06': {'cost': 0.0010250000000000003, 'prompt_tokens': 214, 'completion_tokens': 49, 'total_tokens': 263}}}


{'usage_including_cached_inference': {'total_cost': 0.00034750000000000004, 'gpt-4o-2024-08-06': {'cost': 0.00034750000000000004, 'prompt_tokens': 83, 'completion_tokens': 14, 'total_tokens': 97}}, 'usage_excluding_cached_inference': {'total_cost': 0.00034750000000000004, 'gpt-4o-2024-08-06': {'cost': 0.00034750000000000004, 'prompt_tokens': 83, 'completion_tokens': 14, 'total_tokens': 97}}}


{'usage_including_cached_inference': {'total_cost': 0.0034974999999999997, 'gpt-4o-2024-08-06': {'cost': 0.0034974999999999997, 'prompt_tokens': 415, 'completion_tokens': 246, 'total_tokens': 661}}, 'usage_